# ENGG177 In Class Exercises 
### Lecture 4, Problem 3

Spencer Bertsch

In [2]:
import random
import numpy as np

In [31]:
def flipCoin():
    return random.choice([True,False])

Let's run this with a simulation of 10 million trials

In [43]:
# initialize parameters for the simulation
wins = 0
loses = 0
tosses_list = []
winning_tosses_list = []
TRIALS: int = 1_000_000

for i in range(TRIALS):

    # initialize capital to 2
    capital = 2
    playing = True
    tosses = 0
    while playing is True:
        # increment the number of coin tosses
        tosses += 1
        
        # define the amount of money you want to bet here
        if capital > 5: 
            bet = 10 - capital
        else:
            bet = capital

        if flipCoin() is True:
            # add the new winnings to the capital you already have 
            capital = capital + bet

            # if we win the game, then increment wins and break out of the loop
            if capital >= 10: 
                wins += 1
                playing = False
                winning_tosses_list.append(tosses)

        else: 
            # we pay the casino the amount equal to our bet
            capital = capital - bet

            # if we have run out of money, we lost the game so we record the loss and break 
            if capital <= 0:
                loses += 1
                playing = False
                
    tosses_list.append(tosses)

prob_win: float = round(wins/(loses+wins), 4)*100
print(f'After {TRIALS} trials, we recorded {wins} wins and {loses} loses. The player won {prob_win}% of the time.')

After 1000000 trials, we recorded 199971 wins and 800029 loses. The player won 20.0% of the time.


In [44]:
print(f'Probability of winning is: {round(prob_win)}%')

Probability of winning is: 20%


We can also test our solution to the homework problem using this code: 

In [45]:
print(f'On average, the coin was tossed {round(sum(tosses_list)/len(tosses_list), 1)} times.')

On average, the coin was tossed 2.0 times.


In [46]:
print(f'When the gambler won, the coin was tossed an average of {round(sum(winning_tosses_list)/len(winning_tosses_list), 1)} times.')

When the gambler won, the coin was tossed an average of 3.6 times.


Ok that was fun and we got the correct answer, but now let's solve it the correct way: 

# Homework 1

### Question 3

Let's define the system of linear equations that we need to solve:

$p_2$ = $1/2 \cdot p_{0} + 1/2 \cdot p_{4}$

$p_4$ = $1/2 \cdot p_{0} + 1/2 \cdot p_{8}$

$p_6$ = $1/2 \cdot p_{2} + 1/2 \cdot p_{10}$

$p_8$ = $1/2 \cdot p_{6} + 1/2 \cdot p_{10}$

$p_{10} = 1$

$p_0 = 0$

We can now reorder these equations to write them in the form $AX=b$

0 = 0.5*p0 - 1*p2 + 0.5*p4 + 0*p6 + 0*p8 + 0*p10  
0 = 0.5*p0 + 0*p2 - 1*p4 + 0*p6 + 1/2*p8 + 0*p10  
0 = 0*p0 + 0.5*p2 + 0*p4 - 1*p6 + 0*p8 + 0.5*p10  
0 = 0*p0 + 0*p2 + 0*p4 + 0.5*p6 - 1*p8 + 0.5*p10  
1 = 0*p0 +0*p2 + 0*p4 + 0*p6 + 0*p8 + 1*p10  
0 = 1*p0 + 0*p2 + 0*p4 + 0*p6 + 0*p8 + 0*p10  


In [72]:
a = np.array([
    [0.5, -1, 0.5, 0, 0, 0],  
    [0.5, 0, -1, 0, 0.5, 0],
    [0, 0.5, 0, -1, 0, 0.5],
    [0, 0, 0, 0.5, -1, 0.5],
    [0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0],
    ]) 

In [73]:
a

array([[ 0.5, -1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. , -1. ,  0. ,  0.5,  0. ],
       [ 0. ,  0.5,  0. , -1. ,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0.5, -1. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [75]:
b = np.array([0, 0, 0, 0, 1, 0])
b

array([0, 0, 0, 0, 1, 0])

In [76]:
x = np.linalg.solve(a, b)
x

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

Okay, fantastic. We got the same answer as the simulation showed above. There's a 20% chance of winning if we start from $p_2$, which is the starting point in the example problem. More interestingly, we see the probability of winning given all the different starting points in the game. If we start with zero dollars, then there's a 0% probability of winning because that is an absorbing node in the network. Similarly, if we start with ten dollars, then there is a 100% probability of winning because that is also an absorbing node. If we start in any other intermidiate node, however, we see that the probability of success increases as we choose nodes in the network that are closer to 10. This makes sense because there are fewer coin tosses needed to end up in the (10) absorbing node when starting from $p_6$ or $p_8$. As per this problem's description, we can see that starting in $p_2$ means that we need to traverse the whole network to get to node (10), but we only need to step backwards one step to get to node (0). Now let's move onto the parts of this problem that we see in the first homework involving the hitting times: 

In [96]:
# define transition probability from each node to adjacent nodes
t = (1/2)

# define probability matrix a
a = np.array([
[0, 1, -t, 0, 0, 0],
[0, 0, 1, 0, -t, 0],
[0, -t, 0, 1, 0, 0],
[0, 0, 0, -t, 1, 0],
[1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1],
])
a

array([[ 0. ,  1. , -0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0. , -0.5,  0. ],
       [ 0. , -0.5,  0. ,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  1. ,  0. ],
       [ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ]])

In [99]:
# define b vector
b = np.array([1, 1, 1, 1, 0, 0])

In [100]:
# solve the system of linear equations
x = np.linalg.solve(a, b)
x

array([0., 2., 2., 2., 2., 0.])

So we can see that the expected number of tosses until the gambler either  
acieves his aim OR loses his capital after starting with two dollars is **two toses**. 

# Homework 1 Question 4

0 = -1*π1 + 1/3*π2 + 1/3*π3 + 0*π4 + 1/3*π5 + 0*π6 + 0*π7 + 0*π8  
0 = 1/3*π1 + -1*π2 + 0*π3 + 1/3*π4 + 0*π5 + 1/3*π6 + 0*π7 + 0*π8  
0 = 1/3*π1 + 0*π2 + -1*π3 + 1/3*π4 + 0*π5 + 0*π6 + 1/3*π7 + 0*π8  
0 = 0*π1 + 1/3*π2 + 1/3*π3 + -1*π4 + 0*π5 + 0*π6 + 0*π7 + 1/3*π8  
0 = 1/3*π1 + 0*π2 + 0*π3 + 0*π4 + -1*π5 + 1/3*π6 + 1/3*π7 + 0*π8  
0 = 0*π1 + 1/3*π2 + 0*π3 + 0*π4 + 1/3*π5 + -1*π6 + 0*π7 + 1/3*π8  
0 = 0*π1 + 0*π2 + 1/3*π3 + 0*π4 + 1/3*π5 + 0*π6 + -1*π7 + 1/3*π8  
0 = 0*π1 + 0*π2 + 0*π3 + 1/3*π4 + 0*π5 + 1/3*π6 + 1/3*π7 + -1*π8  
1 = π1 + π2 + π3 + π4 + π5 + π6 + π7 + π8  


In [61]:
a = np.array([
    [-1, (1/3), (1/3), 0, (1/3), 0, 0, 0],  
    [(1/3), -1, 0, (1/3), 0, (1/3), 0, 0],  
    [(1/3), 0, -1, (1/3), 0, 0, (1/3), 0],  
    [0, (1/3), (1/3), -1, 0, 0, 0, (1/3)],  
    [(1/3), 0, 0, 0, -1, (1/3), (1/3), 0],  
    [0, (1/3), 0, 0, (1/3), -1, 0, (1/3)],  
    [0, 0, (1/3), 0, (1/3), 0, -1, (1/3)],  
    # [0, 0, 0, (1/3), 0, (1/3), (1/3), -1], 
    [1, 1, 1, 1, 1, 1, 1, 1]
    ]) 

In [65]:
b = np.array([0, 0, 0, 0, 0, 0, 0, 1])

In [66]:
x = np.linalg.solve(a, b)

In [67]:
x

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

We can see from the above output that the bug would spend exactly 12.5% of the time, or (1/8) of the time in each corner of the cube. This also makes intuitive sense because there's nothing attracting the bug to any one corner of the cube, so as time goes to infinity we could imagine the bug creating a discrete uniform distribution over each of it's possible positions in the cube spending equal amounts of time in each corner. 

Let's try with the toy example from class

In [68]:
a = np.array([
    [-0.3, 0.4],  
    # [0.3, -0.4], 
    [1, 1]
    ]) 

In [69]:
b = np.array([0, 1])

In [71]:
x = np.linalg.solve(a, b)
x

array([0.57142857, 0.42857143])

We can now extend this to the hitting times problem: 

In [90]:
t = (1/3)

a = np.array([
[1,-t, -t,  0, -t,  0,  0,  0],
[-t, 1,  0, -t,  0, -t,  0, 0],
[-t, 0,  1, -t,  0,  0, -t, 0],
[0,-t, -t,  1,  0,  0,  0,  0],
[-t, 0,  0,  0,  1, -t, -t, 0],
[0,-t,  0,  0, -t,  1,  0,  0],
[0, 0, -t,  0, -t,  0,  1,  0],
[0, 0,  0,  0,  0,  0,  0,  1]
])

In [91]:
b = [1, 1, 1, 1, 1, 1, 1, 0]

In [92]:
x = np.linalg.solve(a, b)
x

array([10.,  9.,  9.,  7.,  9.,  7.,  7.,  0.])

We can see from the result that it will take an expected 10 steps  
for the bug to get from the starting corner to the opposite corner of the cube. 